# Import Dependencies

In [8]:
pip install stable-baselines3[extra]

Note: you may need to restart the kernel to use updated packages.Collecting stable-baselines3[extra]
  Using cached stable_baselines3-2.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached gymnasium-1.0.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 699.0 kB/s eta 0:00:02
   ---------------------- ----------------- 0.8/1.4 MB 1.0 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/958.1 kB ? eta -:--:--
   ---------- ----------------

In [3]:
pip install shimmy>=2.0

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Load Environment

In [7]:
env_name = 'CartPole-v1'
# env = gym.make(env_name)

In [9]:

# Initialize the environment with a specified render mode
env = gym.make(env_name, render_mode='human')

episodes = 5
for episode in range(1, episodes + 1):
    state, _ = env.reset()  # Update for newer gym versions
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, truncated, info = env.step(action)
        done = done or truncated  # Combine done and truncated to end the episode
        score += reward

    print('Episode: {} Score: {}'.format(episode, score))

env.close()


C:\Users\user\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 1 Score: 27.0
Episode: 2 Score: 17.0
Episode: 3 Score: 10.0
Episode: 4 Score: 13.0
Episode: 5 Score: 23.0


# Understand The Environment

In [55]:
env.action_space

Discrete(2)

In [57]:
env.action_space.sample()

0

In [59]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [61]:
env.observation_space.sample()

array([ 3.2036905e+00,  2.1871845e+38,  5.9049681e-02, -2.1548565e+38],
      dtype=float32)

# Train RL Model

In [11]:
log_path = os.path.join('Training', 'Logs')

In [13]:
log_path

'Training\\Logs'

In [21]:
# Implementing PPO algorithm
# Agent setup
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, device='cuda')

Using cpu device


In [23]:
model.learn(total_timesteps= 20000)

Logging to Training\Logs\PPO_1


C:\Users\user\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


-----------------------------
| time/              |      |
|    fps             | 422  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 303         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007820637 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.013       |
|    learning_rate        | 0.0003      |
|    loss                 | 6.8         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0155     |
|    value_loss           | 60.1        |
-----------------------------------------
----------------------------------

# Save And Reload Model

In [28]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')
model.save(PPO_Path)

In [30]:
# delete model
del model

In [32]:
PPO_Path

'Training\\Saved Models\\PPO_Model_Cartpole'

In [34]:
# reload the model
model = PPO.load(PPO_Path, env=env)

# Evaluation

In [37]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\user\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:244: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(485.3, 30.013496963866107)

# Test Model

In [50]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()  # Update for newer gym versions
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)  # using model
        obs, reward, done, info = env.step(action)
        done = done or truncated  # Combine done and truncated to end the episode
        score += reward

    print('Episode: {} Score: {}'.format(episode, score))

env.close()


Episode: 1 Score: [500.]
Episode: 2 Score: [500.]
Episode: 3 Score: [402.]
Episode: 4 Score: [206.]
Episode: 5 Score: [447.]
